# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [3]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [4]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [5]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [2]:
import pandas as pd
from xml.etree import ElementTree as ET
import numpy as np


document = ET.parse( './data/mondial_database.xml' )
root = document.getroot()

In [4]:
# Temp Data Frame 
country = ""
inf_mor = ""
d_frame = pd.DataFrame(columns = ["country","inf_mor"])
d_frame['inf_mor'] = d_frame['inf_mor'].astype(float)
d_frame

,country,inf_mor


In [9]:
# Get 10 countries with lowest infant mortality rates

for country in document.findall( 'country' ):
    print (country.tag,country.attrib)

country {'capital': 'cty-Albania-Tirane', 'memberships': 'org-BSEC org-CEI org-CD org-SELEC org-CE org-EAPC org-EBRD org-EITI org-FAO org-IPU org-IAEA org-IBRD org-ICC org-ICAO org-ICCt org-Interpol org-IDA org-IFRCS org-IFC org-IFAD org-ILO org-IMO org-IMF org-IOC org-IOM org-ISO org-OIF org-ITU org-ITUC org-IDB org-MIGA org-NATO org-OSCE org-OPCW org-OAS org-OIC org-PCA org-UN org-UNCTAD org-UNESCO org-UNIDO org-UPU org-WCO org-WFTU org-WHO org-WIPO org-WMO org-UNWTO org-WTO', 'car_code': 'AL', 'area': '28750'}
country {'capital': 'cty-Greece-Athens', 'memberships': 'org-AG org-BIS org-BSEC org-CD org-SELEC org-CE org-EMU org-EAPC org-EBRD org-ECB org-EIB org-CERN org-ESA org-EU org-FATF org-FAO org-IGAD org-IPU org-IAEA org-IBRD org-ICC org-ICAO org-ICJ org-ICCt org-Interpol org-IDA org-IEA org-IFRCS org-IFC org-IFAD org-IHO org-ILO org-IMO org-IMSO org-IMF org-IOC org-IOM org-OIF org-ITSO org-ITU org-ITUC org-MIGA org-NATO org-NEA org-NSG org-OECD org-OSCE org-OPCW org-OAS org-PCA 

In [11]:
# Creating a DataFrame for easier sorting later
country_name = ""
infant_mortality = ""
d_frame = pd.DataFrame(columns = ["country_name","infant_mortality"])
d_frame['infant_mortality'] = d_frame['infant_mortality'].astype(float)
d_frame

for country in document.findall( 'country' ):
    for node in country.getiterator(): 
        if node.tag == 'name': #find country name tag
            if country_name == "":
                country_name = (node.text)
        if node.tag == 'infant_mortality': #find mortality rate tag
            infant_mortality = float(node.text)
    d_frame.loc[len(d_frame)] = [country_name,infant_mortality] #add country name and mortality rate to data frame
    country_name = ""
    
    
#sort data frame and find top 10 countries with lowest mortalitiy rates
d_frame.sort_values(by = 'infant_mortality').head(10)

,country_name,infant_mortality
38,Monaco,1.81
98,Japan,2.13
36,Norway,2.48
117,Bermuda,2.48
106,Singapore,2.53
37,Sweden,2.60
10,Czech Republic,2.63
78,Hong Kong,2.73
79,Macao,3.13
44,Iceland,3.15


In [ ]:
# top 10 Cities with largest population

In [15]:
cityname = ""
citypopulation = ""
document = ET.parse( './data/mondial_database.xml' )
df = pd.DataFrame(columns=['CityName','Population']) #create data frame to hold country name and its popuplation
df['Population'] = df['Population'].astype(float)

#loop through country element to find city name and its population
for country in document.iterfind( 'country' ):
    for city in country.iter('city'): #find all cities within each country element
        cityname = city.find('name').text
        year = int(0)
        for node in city.iterfind('population'): #find all population elements with each city
            year = node.attrib['year'] #there are multiple population elements with different 'year' attribute
            if node.attrib['year'] >= year: #store the population number of the latest year
                citypopulation = int(node.text)
        df.loc[len(df)] = [cityname,citypopulation] #add city name and its population to data frame
        cityname = ""
        

#sort data frame to find 10 cities with largest population
df.sort_values(by = 'Population', ascending=False).head(10)

,CityName,Population
1341,Shanghai,22315474.0
771,Istanbul,13710512.0
1527,Mumbai,12442373.0
479,Moskva,11979529.0
1340,Beijing,11716620.0
2810,São Paulo,11152344.0
1342,Tianjin,11090314.0
1064,Guangzhou,11071424.0
1582,Delhi,11034555.0
1067,Shenzhen,10358381.0


In [ ]:
# 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)


In [16]:
# 3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
countryname = ""
countrypopulation = 0
ethnicpopulation = 0
document = ET.parse( './data/mondial_database.xml' )
df = pd.DataFrame(columns=['Country','EthnicGroup','Population'])
df['Population'] = df['Population'].astype(float)

#loop through country element to find ethnic groups and its population
for country in document.iterfind( 'country' ):
    countryname = country.find('name').text #find country name
    year = int(0)
    countrycpopulation = int(0)
    for node in country.iterfind('population'): #find population of the country
        year = node.attrib['year']
        if node.attrib['year'] >= year: #find population of the latest year
            countrypopulation = (node.text) 
    ethnicname = None
    ethnicpopulation = 0
    for ethnic in country.iter('ethnicgroup'): #find all ethnic groups within the same country
        ethnicname = ethnic.text
        #compute each ethnic population: country population * ethnic group percentage
        ethnicpopulation = round(float(ethnic.attrib['percentage']) * 0.01 * int(countrypopulation))
        if ethnicname == None:
            ethnicname = countryname
            ethnicpopulation = countrypopulation
        df.loc[len(df)] = [countryname,ethnicname,ethnicpopulation] #store ethnic group population to data frame
    countryname = ""

#group ethnic group across all countries and sum them up to find top 10 ethnic groups and its total population
df.groupby('EthnicGroup').sum().sort_values(by = 'Population', ascending=False).head(10)

,Population
EthnicGroup,
Han Chinese,1.245059e+09
Indo-Aryan,8.718156e+08
European,4.948722e+08
African,3.183251e+08
Dravidian,3.027137e+08
Mestizo,1.577344e+08
Bengali,1.467769e+08
Russian,1.318570e+08
Japanese,1.265342e+08


In [ ]:
# name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [17]:
# name and country of a) longest river
river_dict = {}
dfcc = pd.DataFrame.from_dict(country_dict, orient='index').reset_index().set_index('car_code')

for riv in root.iter('river'):
    river = {}
    rid = riv.attrib['id']
    river['name'] = riv.find('name').text
    river['ccodes'] = riv.attrib['country']
    
    try:
        river['length'] = riv.find('length').text
    except:
        river['length'] = np.nan
        
    
    country_names = []
    for cc in riv.attrib['country'].split(' '):
        country_names.append(dfcc.loc[cc]['index'])
    
    river['country_names'] = country_names
    river_dict[rid] = river
    
dfr = pd.DataFrame.from_dict(river_dict, orient='index')
dfr.length = dfr.length.astype(float)
dfr.sort_values(by='length', ascending=False).head(1)

NameError: name 'country_dict' is not defined

In [18]:
# Name and Country of b) largest lake
# TODO: Write a function to generate these dicts.

lake_dict = {}
for lk in root.iter('lake'):
    lake = {}
    lid = lk.attrib['id']
    lake['name'] = lk.find('name').text
    lake['ccodes'] = lk.attrib['country']
    
    try:
        lake['area'] = float(lk.find('area').text)
    except:
        lake['area'] = np.nan
        
    
    country_names = []
    for cc in lk.attrib['country'].split(' '):
        country_names.append(dfcc.loc[cc]['index'])
    
    lake['country_names'] = country_names
    lake_dict[lid] = lake
    
dflk = pd.DataFrame.from_dict(lake_dict, orient='index')
dflk.area = dflk.area.astype(float)
dflk.sort_values(by='area', ascending=False).head(10)[['name', 'area', 'country_names']]

NameError: name 'dfcc' is not defined

In [19]:
# Name and Country of c) airport at highest elevation
# TODO: Write a function to generate these dicts.

air_dict = {}
for ap in root.iter('airport'):
    airport = {}
    aid = ap.attrib['iatacode']
    airport['name'] = ap.find('name').text
    airport['ccodes'] = ap.attrib['country']
    
    try:
        airport['elevation'] = float(ap.find('elevation').text)
    except:
        airport['elevation'] = np.nan
        
    
    country_names = []
    for cc in ap.attrib['country'].split(' '):
        country_names.append(dfcc.loc[cc]['index'])
    
    airport['country_names'] = country_names
    air_dict[aid] = airport

dfair = pd.DataFrame.from_dict(air_dict, orient='index')
dfair.area = dfair.elevation.astype(float)
dfair.sort_values(by='elevation', ascending=False).head(10)[['name', 'elevation', 'country_names']]

NameError: name 'dfcc' is not defined